In [1]:
import time
import re, string
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as pat
import pandas as pd
from collections import Counter

import scipy as sp
import scipy.cluster
import scipy.cluster.hierarchy as hierarchy
import scipy.spatial.distance

from sklearn.cluster import KMeans
import sklearn.datasets as sk_data
import sklearn.metrics as metrics
from sklearn import mixture
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.utils.extmath import randomized_svd

#import matplotlib as mpl
import seaborn as sns

import nltk
#nltk.download()
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
%matplotlib inline


In [2]:
# Extracts data from DISP_AllStatesAndTerritories.xlsx
xl = pd.ExcelFile("DISP_AllStatesAndTerritories.xlsx")
states = xl.sheet_names                                  # Creates states, which is a list of name os states (tabs)

In [4]:
for tab in states:                    # Creates a loop based on states
    file_type = '.pkl'                # Generates a file type .pkl
    file_name = tab + file_type       # Create file name base off each state
    df = xl.parse(tab)                # Parses each state out of the xl dataframe
    df.to_pickle(file_name)           # Output the each file into a pickle file of each state

In [5]:
df

,State,Station Name (LEA),NSN,Item Name,Quantity,UI,Acquisition Value,DEMIL Code,DEMIL IC,Ship Date
0,VI,VIRGIN ISLANDS POLICE DEPT,8430-DS-BOO-TS00,"BOOT, MEN'S,",5,Each,40.00,A,NaN,2017-03-16
1,VI,VIRGIN ISLANDS POLICE DEPT,4020-00-689-5684,"ROPE,FIBROUS",4,Coil,118.64,A,1.0,2017-04-12
2,VI,VIRGIN ISLANDS POLICE DEPT,8465-01-524-8407,"CARRIER,INTRENCHING TOOL",24,Each,19.80,A,1.0,2017-03-16
3,VI,VIRGIN ISLANDS POLICE DEPT,8345-00-682-6858,"FLAG,NATIONAL",2,Each,174.10,A,NaN,2017-03-16
4,VI,VIRGIN ISLANDS POLICE DEPT,8430-DS-BOO-TS00,"BOOT, MEN'S,",1,Each,40.00,A,NaN,2017-03-16
5,VI,VIRGIN ISLANDS POLICE DEPT,2540-01-532-3231,TARPAULIN,3,Each,113.60,A,1.0,2017-03-16
6,VI,VIRGIN ISLANDS POLICE DEPT,2540-01-532-3231,TARPAULIN,21,Each,113.60,A,1.0,2017-03-16
7,VI,VIRGIN ISLANDS POLICE DEPT,2320-01-107-7155,"TRUCK,UTILITY",1,Each,41061.00,Q,6.0,2012-06-12
8,VI,VIRGIN ISLANDS POLICE DEPT,7690-00-717-6320,"MARKER,IDENTIFICATION",4,Roll,30.71,A,1.0,2017-03-16
9,VI,VIRGIN ISLANDS POLICE DEPT,1005-00-589-1271,"RIFLE,7.62 MILLIMETER",1,Each,138.00,D,1.0,1996-08-20
